# Implementing a simple Decision Tree

###### "The goal in learning is to figure out what questions to ask, in what order to ask them, and what answer to predict once you have asked enough questions" [1]

## Libraries

In [479]:
import numpy as np
import pandas as pd

## Representing the nodes and leafs of the tree

In [480]:
class Leaf:
    
    def __init__(self, guess):
        
        self.guess = guess

In [481]:
class Node:
    
    def __init__(self, f, l, r):
        
        self.feature = f
        self.left = l
        self.right = r

## The Decision Tree

In [482]:
class DecisionTree:
    
    def __init__(self, X):
        
        self.default = np.bincount(X.iloc[:,-1].to_numpy()).argmax()
    
    def train(self, X, features):
        
        y = X.iloc[:,-1].to_numpy()
        if len(y) == 0:
            return Leaf(self.default)
        
        guess = np.bincount(y).argmax()
        
        if len(np.unique(y)) == 1:
            return Leaf(guess)
        
        elif len(features) == 0:
            return Leaf(guess)
        
        else:

            scores = dict()

            for f in features:
                
                data_no = X[X[f] == 0]
                data_yes = X[X[f] == 1]
                
                if len(data_no) != 0:
                    score_no = np.max(np.bincount(data_no.iloc[:,-1].to_numpy()))
                else:
                    score_no = 0
                    
                if len(data_yes) != 0:
                    score_yes = np.max(np.bincount(data_yes.iloc[:,-1].to_numpy()))
                else:
                    score_yes = 0
                
                scores[f] = score_no + score_yes
                            
            chosen_feature = max(scores, key=scores.get)
            data_no = X[X[chosen_feature] == 0]
            data_yes = X[X[chosen_feature] == 1]
            
            new_features = features.copy()
            new_features.remove(chosen_feature)
            
            left = self.train(data_no, new_features)
            right = self.train(data_yes, new_features)

            return Node(chosen_feature, left, right)
        
    def test(self, node, test_data):

        if isinstance(node, Leaf):
            return node.guess
        elif isinstance(node, Node):
            if test_data[node.feature].values[0] == 0:
                return self.test(node.left, test_data)
            else:
                return self.test(node.right, test_data)

## Creating a random dataset

In [483]:
dataset = np.random.randint(0, 2, size=(40, 5))

In [484]:
target = np.array([0]*19 + [1]*21).reshape(40, 1)

dataset = np.concatenate((dataset, target), axis=1)

In [485]:
features = ['f1', 'f2', 'f3', 'f4', 'f5']
columns_named = features + ['target']

In [486]:
df = pd.DataFrame(dataset, columns=columns_named)
df

,f1,f2,f3,f4,f5,target
0,0,1,1,0,0,0
1,1,1,1,1,1,0
2,1,1,0,0,1,0
3,1,0,1,0,0,0
4,1,1,1,0,1,0
5,1,0,0,0,0,0
6,1,0,1,0,1,0
7,0,0,1,1,0,0
8,1,1,0,1,1,0
9,1,0,1,0,0,0


## Training with the data

In [487]:
model = DecisionTree(df)

In [488]:
root = model.train(df, features)

In [489]:
root

## Visualizing the tree

In [490]:
def print_tree(node, depth=0):
    if isinstance(node, Leaf):
        print("  " * depth + "Guess:", node.guess)
    elif isinstance(node, Node):
        print("  " * depth + "Feature:", node.feature)
        print("  " * depth + "Left:")
        print_tree(node.left, depth + 1)
        print("  " * depth + "Right:")
        print_tree(node.right, depth + 1)


In [491]:
print_tree(root)

Feature: f4
Left:
  Feature: f1
  Left:
    Feature: f2
    Left:
      Guess: 0
    Right:
      Feature: f3
      Left:
        Feature: f5
        Left:
          Guess: 0
        Right:
          Guess: 0
      Right:
        Feature: f5
        Left:
          Guess: 1
        Right:
          Guess: 1
  Right:
    Feature: f2
    Left:
      Feature: f3
      Left:
        Guess: 0
      Right:
        Feature: f5
        Left:
          Guess: 0
        Right:
          Guess: 1
    Right:
      Feature: f3
      Left:
        Feature: f5
        Left:
          Guess: 1
        Right:
          Guess: 0
      Right:
        Feature: f5
        Left:
          Guess: 1
        Right:
          Guess: 0
Right:
  Feature: f1
  Left:
    Feature: f2
    Left:
      Feature: f3
      Left:
        Guess: 1
      Right:
        Feature: f5
        Left:
          Guess: 0
        Right:
          Guess: 1
    Right:
      Feature: f3
      Left:
        Guess: 1
      Right:
        

## Classifying a new instance

In [492]:
df_test = pd.DataFrame(np.array([1,0,1,1,0]).reshape(1,5), columns=features[:5])

In [493]:
df_test

,f1,f2,f3,f4,f5
0,1,0,1,1,0


In [494]:
model.test(root, df_test)
print(model.test(root, df_test))

1


## References

-	
[1] A Course in Machine Learning, by Hal Daumé III. Decisions Trees: http://ciml.info/dl/v0_99/ciml-v0_99-ch01.pdf